In [0]:


# Initialize Spark session
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("YourAppName").getOrCreate()

In [0]:
%pip install -r requirements.txt

In [0]:
dbutils.library.restartPython()

In [0]:
# flight_simulator.py
import sqlite3
import random
from datetime import datetime, timedelta
# from apscheduler.schedulers.background import BackgroundScheduler
import time
import os

DB_FILE = "flight_status.db"

def create_sample_flight_data():
    airlines = ["Delta", "American", "United", "JetBlue"]
    airports = ["JFK", "LAX", "ORD", "DFW", "ATL", "SEA", "MIA"]
    statuses = ["ON TIME", "DELAYED", "CANCELLED", "IN AIR"]
    now = datetime.now().replace(second=0, microsecond=0)

    flights = []
    for i in range(10):
        airline = random.choice(airlines)
        origin = random.choice(airports)
        destination = random.choice([ap for ap in airports if ap != origin])
        departure = now + timedelta(minutes=random.randint(-60, 180))
        arrival = departure + timedelta(hours=random.uniform(1.0, 5.0))
        status = random.choice(statuses)
        gate = f"{random.choice(['A', 'B', 'C'])}{random.randint(1, 20)}"
        flights.append((
            f"{airline[:2].upper()}00{(i)}",
            airline,
            origin,
            destination,
            departure.isoformat(),
            arrival.isoformat(),
            status,
            gate
        ))
    return flights

def init_db():
    if os.path.exists(DB_FILE):
        os.remove(DB_FILE)

    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE flights (
            flight_id TEXT,
            airline TEXT,
            origin TEXT,
            destination TEXT,
            sched_departure TEXT,
            sched_arrival TEXT,
            status TEXT,
            gate TEXT
        )
    """)
    cursor.executemany("""
        INSERT INTO flights VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, create_sample_flight_data())
    conn.commit()
    conn.close()
    print("Initialized flight_status.db with mock flight data.")

def update_random_flight_status():
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()

    cursor.execute("SELECT flight_id FROM flights ORDER BY RANDOM() LIMIT 1")
    result = cursor.fetchone()
    if not result:
        return
    flight_id = result[0]

    new_status = random.choice(["ON TIME", "DELAYED", "CANCELLED", "IN AIR"])
    cursor.execute("""
        UPDATE flights
        SET status = ?, sched_departure = ?
        WHERE flight_id = ?
    """, (new_status, datetime.now().isoformat(), flight_id))

    conn.commit()

    cursor.execute("SELECT * FROM flights WHERE flight_id = ?", (flight_id,))
    flight = cursor.fetchone()
    conn.close()

    print(f"🛩️ [{datetime.now().strftime('%H:%M:%S')}] Updated Flight {flight[0]} → Status: {flight[6]}")

# def start_scheduler():
#     scheduler = BackgroundScheduler()
#     scheduler.add_job(update_random_flight_status, 'interval', seconds=10)
#     scheduler.start()
#     print("🚀 Live update scheduler started (every 10s).")


In [0]:
init_db()
# start_scheduler()

# try:
#     while True:
#         time.sleep(60)  # Keeps the script alive
# except (KeyboardInterrupt, SystemExit):
#     print("Scheduler stopped.")

In [0]:
from databricks_langchain import ChatDatabricks
from databricks.sdk import WorkspaceClient
import os

w = WorkspaceClient()

os.environ["DATABRICKS_HOST"] = w.config.host
os.environ["DATABRICKS_TOKEN"] = w.tokens.create(comment="for model serving", lifetime_seconds=1200).token_value

llm = ChatDatabricks(endpoint="databricks-llama-4-maverick")

In [0]:
llm.invoke("hello, world")

In [0]:
from langchain.agents import initialize_agent, Tool
from langchain.tools import tool
from langchain.agents.agent_types import AgentType
# from langchain_anthropic import ChatAnthropic
import sqlite3
import os

DB_PATH = "flight_status.db"

@tool
def query_flight_schedule(sql: str) -> str:
    """Run a SQL query on the flight_status.db file."""
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute(sql)
        rows = cursor.fetchall()
        col_names = [description[0] for description in cursor.description]
        conn.close()
        return "\n".join([str(dict(zip(col_names, row))) for row in rows]) or "No results."
    except Exception as e:
        return f"SQL Error: {str(e)}"

tools = [
    Tool.from_function(
        func=query_flight_schedule,
        name="query_flight_schedule",
        description="Run a SQL query on the flight schedule DB to get information about flights, gates, times, delays, etc."
    )
]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

if __name__ == "__main__":
    response = agent.run(
        "Check for any risky issues in the next 2 hours of scheduled flights: gate conflicts, delays, or congestion."
    )
    print("📋 Agent Report:")
    print(response)
